# Imports

In [24]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import warnings
import data_func.read_data as read_data
warnings.filterwarnings("ignore", category=FutureWarning, module="catboost")



# Load datasets

In [25]:
dataframes = read_data.get_training_data()
X_frames_train = dataframes[0]
Y_frames_train = dataframes[1]
X_frames_test = read_data.get_test_data()


# Data clean up

In [26]:
# Making sure that target values line up with x_values

def data_allign(x_train, y_train):
  
  y_train.dropna(inplace=True)
  combined_data = pd.merge(x_train, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    
  return combined_data, y_train

import data_func.aggregation as data_agg

categorical_features = ['date_forecast', 'dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx', 'precip_type_5min:idx']
categorical_features_to_drop = ['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx', 'precip_type_5min:idx']

for i in range(len(X_frames_train)):
  categorical_frame = X_frames_train[i][categorical_features]
  categorical_frame = data_agg.gen_agg(categorical_frame, data_agg.stocastic_median)

  X_frames_train[i] = X_frames_train[i].drop(columns=categorical_features_to_drop)
  X_frames_train[i] = data_agg.gen_agg(X_frames_train[i], 'mean')
  X_frames_train[i] = pd.merge(X_frames_train[i], categorical_frame, on='date_forecast')
  X_frames_train[i], Y_frames_train[i] = data_allign(X_frames_train[i], Y_frames_train[i])

for j in range(len(X_frames_test)):
  categorical_frame = X_frames_test[i][categorical_features]
  categorical_frame = data_agg.gen_agg(categorical_frame, data_agg.stocastic_median)

  X_frames_test[i] = X_frames_test[i].drop(columns=categorical_features_to_drop)
  X_frames_test[i] = pd.merge(X_frames_test[i], categorical_frame, on='date_forecast')
  X_frames_test[j] = data_agg.gen_agg(X_frames_test[j], 'mean')



# Feature engineering

In [27]:
import data_func.timeseasonality as DTS
import data_func.date_forecast as DTF
import data_func.one_hot_encoding as OHE

categorical_features_to_one_hot = ['dew_or_rime:idx', 'precip_type_5min:idx']

for i in range(len(X_frames_train)):
    X_frames_train[i] = DTS.append_seasonal_columns(X_frames_train[i])
    X_frames_train[i] = DTF.date_forecast_columns(X_frames_train[i])
    # X_frames_train[i] = OHE.one_hot_encode(X_frames_train[i], categorical_features_to_one_hot)
    X_frames_train[i].drop(columns=['snow_drift:idx'], inplace=True)

    # X_frames_train[i].drop(columns=['absolute_humidity_2m:gm3'], inplace=True)
    # X_frames_train[i].drop(columns=['air_density_2m:kgm3'], inplace=True)
    # X_frames_train[i]['ceiling_height_agl:m'] = X_frames_train[i]['ceiling_height_agl:m'].fillna(0)
    # X_frames_train[i]['cloud_base_agl:m'] = X_frames_train[i]['cloud_base_agl:m'].fillna(100000)



for i in range(len(X_frames_test)):
    X_frames_test[i] = DTS.append_seasonal_columns(X_frames_test[i])
    X_frames_test[i] = DTF.date_forecast_columns(X_frames_test[i])
    # X_frames_test[i] = OHE.one_hot_encode(X_frames_test[i], categorical_features_to_one_hot)




# Hyperparameter optimization

In [28]:
import optuna
import catboost as cat # Change to model to optimize

# Split the data into training and validation sets
x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(X_frames_train[0], Y_frames_train[0], test_size=0.17, random_state=None)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(X_frames_train[1], Y_frames_train[1], test_size=0.17, random_state=None)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(X_frames_train[2], Y_frames_train[2], test_size=0.17, random_state=None)


def objective(trial):
    
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "cat_features": categorical_features_to_drop,
        "random_seed": 42,
        "loss_function": "MAE",
        "eval_metric": "MAE",
    }

    model_c = cat.CatBoostRegressor(**params) # Change to model to optimize
    model_c.fit(x_train_c, y_train_c)

    # Make predictions on the validation set
    y_pred = model_c.predict(x_val_c)

    # Calculate the Mean Squared Error (MSE) as the metric to optimize
    mae = mean_absolute_error(y_val_c, y_pred)

    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best = study.best_params
# Print the best hyperparameters found
print("Best hyperparameters:", best)

[I 2023-10-30 10:36:36,270] A new study created in memory with name: no-name-bed4b793-c011-44f1-b0a7-d54a44143018


0:	learn: 80.2776363	total: 9.59ms	remaining: 9.58s
1:	learn: 77.3581000	total: 15.6ms	remaining: 7.8s
2:	learn: 74.4293077	total: 25ms	remaining: 8.32s
3:	learn: 71.8788530	total: 30.2ms	remaining: 7.52s
4:	learn: 70.0388602	total: 35.9ms	remaining: 7.13s
5:	learn: 67.6543190	total: 44.2ms	remaining: 7.32s
6:	learn: 64.7843777	total: 49.7ms	remaining: 7.04s
7:	learn: 64.7641159	total: 72.3ms	remaining: 8.96s
8:	learn: 64.7641159	total: 76.4ms	remaining: 8.41s
9:	learn: 62.5519747	total: 80.9ms	remaining: 8.01s
10:	learn: 60.7243693	total: 85.9ms	remaining: 7.72s
11:	learn: 58.7608853	total: 98.3ms	remaining: 8.09s
12:	learn: 57.4643564	total: 105ms	remaining: 7.96s
13:	learn: 56.1402986	total: 112ms	remaining: 7.88s
14:	learn: 53.9997679	total: 117ms	remaining: 7.66s
15:	learn: 52.3731404	total: 129ms	remaining: 7.92s
16:	learn: 51.4499477	total: 133ms	remaining: 7.69s
17:	learn: 50.2823720	total: 140ms	remaining: 7.62s
18:	learn: 49.0488346	total: 146ms	remaining: 7.55s
19:	learn: 47

[I 2023-10-30 10:36:43,576] Trial 0 finished with value: 21.449755781609742 and parameters: {'learning_rate': 0.058625673355017845, 'depth': 4, 'subsample': 0.12085282233486801, 'colsample_bylevel': 0.09265478612536795, 'min_data_in_leaf': 44}. Best is trial 0 with value: 21.449755781609742.


983:	learn: 22.1975102	total: 6.91s	remaining: 112ms
984:	learn: 22.1972185	total: 6.91s	remaining: 105ms
985:	learn: 22.1972101	total: 6.92s	remaining: 98.3ms
986:	learn: 22.1950199	total: 6.94s	remaining: 91.4ms
987:	learn: 22.1755164	total: 6.95s	remaining: 84.4ms
988:	learn: 22.1754703	total: 6.95s	remaining: 77.3ms
989:	learn: 22.1750993	total: 6.96s	remaining: 70.3ms
990:	learn: 22.1750954	total: 6.96s	remaining: 63.2ms
991:	learn: 22.1738634	total: 6.97s	remaining: 56.2ms
992:	learn: 22.1738431	total: 6.97s	remaining: 49.2ms
993:	learn: 22.1738199	total: 6.98s	remaining: 42.2ms
994:	learn: 22.1732554	total: 6.99s	remaining: 35.1ms
995:	learn: 22.1732531	total: 7s	remaining: 28.1ms
996:	learn: 22.1732341	total: 7.01s	remaining: 21.1ms
997:	learn: 22.1715668	total: 7.02s	remaining: 14.1ms
998:	learn: 22.1711454	total: 7.02s	remaining: 7.03ms
999:	learn: 22.1711324	total: 7.04s	remaining: 0us
0:	learn: 78.6288757	total: 42.6ms	remaining: 42.6s
1:	learn: 75.1534333	total: 90.7ms	rem

[I 2023-10-30 10:37:30,128] Trial 1 finished with value: 17.60753052796567 and parameters: {'learning_rate': 0.060183973978509046, 'depth': 10, 'subsample': 0.28819995411353627, 'colsample_bylevel': 0.39818319507473743, 'min_data_in_leaf': 77}. Best is trial 1 with value: 17.60753052796567.


...

977:	learn: 25.1218496	total: 4.59s	remaining: 103ms
978:	learn: 25.1210565	total: 4.59s	remaining: 98.6ms
979:	learn: 25.1209874	total: 4.6s	remaining: 93.8ms
980:	learn: 25.1208992	total: 4.6s	remaining: 89.1ms
981:	learn: 25.1189456	total: 4.61s	remaining: 84.4ms
982:	learn: 25.1189453	total: 4.61s	remaining: 79.7ms
983:	learn: 25.1163328	total: 4.61s	remaining: 75ms
984:	learn: 25.1142396	total: 4.62s	remaining: 70.3ms
985:	learn: 25.1142394	total: 4.62s	remaining: 65.6ms
986:	learn: 25.1142394	total: 4.63s	remaining: 60.9ms
987:	learn: 25.1124903	total: 4.63s	remaining: 56.2ms
988:	learn: 25.1024645	total: 4.63s	remaining: 51.6ms
989:	learn: 25.1004962	total: 4.64s	remaining: 46.9ms
990:	learn: 25.0959885	total: 4.64s	remaining: 42.2ms
991:	learn: 25.0959881	total: 4.64s	remaining: 37.5ms
992:	learn: 25.0959881	total: 4.65s	remaining: 32.8ms
993:	learn: 25.0903843	total: 4.66s	remaining: 28.1ms
994:	learn: 25.0903811	total: 4.66s	remaining: 23.4ms
995:	learn: 25.0903537	total: 4.6

[I 2023-10-30 10:42:30,787] Trial 8 finished with value: 20.335258654082057 and parameters: {'learning_rate': 0.011641240541167342, 'depth': 6, 'subsample': 0.2939597767927058, 'colsample_bylevel': 0.33935343589213207, 'min_data_in_leaf': 52}. Best is trial 1 with value: 17.60753052796567.


996:	learn: 20.9824658	total: 23.5s	remaining: 70.8ms
997:	learn: 20.9812066	total: 23.6s	remaining: 47.2ms
998:	learn: 20.9777855	total: 23.6s	remaining: 23.6ms
999:	learn: 20.9776776	total: 23.6s	remaining: 0us
0:	learn: 82.6013154	total: 70.5ms	remaining: 1m 10s
1:	learn: 82.2489557	total: 125ms	remaining: 1m 2s
2:	learn: 81.9681007	total: 185ms	remaining: 1m 1s
3:	learn: 81.6045788	total: 241ms	remaining: 1m
4:	learn: 81.3072912	total: 300ms	remaining: 59.8s
5:	learn: 80.9820662	total: 366ms	remaining: 1m
6:	learn: 80.6712300	total: 426ms	remaining: 1m
7:	learn: 80.3579063	total: 491ms	remaining: 1m
8:	learn: 80.0682692	total: 554ms	remaining: 1m 1s
9:	learn: 79.7543095	total: 621ms	remaining: 1m 1s
10:	learn: 79.4319735	total: 686ms	remaining: 1m 1s
11:	learn: 79.1524418	total: 746ms	remaining: 1m 1s
12:	learn: 78.8799582	total: 806ms	remaining: 1m 1s
13:	learn: 78.6062575	total: 868ms	remaining: 1m 1s
14:	learn: 78.3355050	total: 923ms	remaining: 1m
15:	learn: 78.0069643	total: 9

[I 2023-10-30 10:43:30,398] Trial 9 finished with value: 19.84705936045429 and parameters: {'learning_rate': 0.005040034669162006, 'depth': 10, 'subsample': 0.16883178170052698, 'colsample_bylevel': 0.5939332139971534, 'min_data_in_leaf': 67}. Best is trial 1 with value: 17.60753052796567.


Best hyperparameters: {'learning_rate': 0.060183973978509046, 'depth': 10, 'subsample': 0.28819995411353627, 'colsample_bylevel': 0.39818319507473743, 'min_data_in_leaf': 77}
